# Week 14: Poly Cube Mapping Attempt
A naive test on poly-cube mapping

## Table of Contents
- [0 - Packages and Resources](#0)
- [1 - Implementation](#1)
    - [*1.0 - The Example](#1-0)
    - [1.1 - Area Calculation](#1-1)
    - [1.2 - BFS](#1-2)
    - [1.3 - Fit a Plane](#1-3)
    - [1.4 - Intersection of Plane and Mesh](#1-4)
- [2 - Sample Outcome](#2)
    - [2.1 - One Round](#2-1)
    - [2.2 - Convergence](#2-2)
- [3 - Reference](#3)

<a name='0'></a>
## 0. Packages and Resources

In [2]:
# Packages
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util.util import distance_euclidean
from util.mesh.triangle.R3 import calculate_single_dihedral_angle, calculate_all_dihedral_angles, calculate_face_normals
from util.mesh.triangle.common import retrieve_all_edges
from util.triangle import calculate_area

# Visualization
import pyvista as pv
from pyvista import examples

<a name='1'></a>
## 1. Implementation

<a name='1-0'></a>
### *1.0 The Example

Here I use the sphere as an example.

In [3]:
sphere = pv.Sphere(radius=5, theta_resolution=10, phi_resolution=10)

faces = sphere.faces.reshape((-1,4))[:, 1:4]
vertices = sphere.points
angles = calculate_all_dihedral_angles(faces, vertices)
edges = retrieve_all_edges(faces)

pl = pv.Plotter()
pl.add_mesh(sphere, show_edges=True, color="white")
pl.add_points(sphere.points, color="red", point_size=5)
# For monitor purpose
# pl.add_points(sphere.points[55], color="blue", point_size=20)
# pl.add_points(sphere.points[64], color="blue", point_size=20)
pl.show()

Widget(value="<iframe src='http://localhost:37593/index.html?ui=P_0x7f85e84503d0_0&reconnect=auto' style='widt…

In [4]:
class chen_2023:
    def __init__(self, vertices, faces, lambda1=1, lambda2=1, max_iter = 30):
        self.max_iter = max_iter
        self.vertex_num = vertices.shape[0]
        self.vertices = vertices
        self.faces = faces
        self.lambda1=lambda1
        self.lambda2=lambda2
        
        self.solution = self.vertices.copy()
        self.edges = retrieve_all_edges(faces)
    
    def activation(self, angle):
        if angle < (np.pi/2):
            return np.power(np.cos(angle), 2) / angle
        else:
            return np.power(np.cos(angle), 2)
        
    def loss_classification(self, x):
        X = x.reshape((self.vertex_num , 3))
        
        EB = 0
        for i, face in enumerate(self.faces):
            v1, v2, v3 = X[face[0]], X[face[1]], X[face[2]]
            a = v2 - v1
            b = v3 - v1
            cross_product = np.cross(a, b)
#             normal = cross_product / np.linalg.norm(cross_product)
            area = np.linalg.norm(cross_product) / 2
#             EB += area * (np.abs(normal) - 1)
            EB += np.sum(np.abs(cross_product)) - area
#         print(EB)
            
        EA = 0
        dihedral_angles = calculate_all_dihedral_angles(faces, X)
        for angle_value in dihedral_angles.values():
            EA += self.activation(angle_value)
            
        return self.lambda1*EB + self.lambda2*EA
    
    def optimize(self):
        x0 = np.ravel(self.solution)
        self.res = minimize(self.loss_classification, x0, options = {'maxiter': self.max_iter})
        self.solution = self.res.x.reshape((self.vertex_num, 3))
    
    def optimize_one_round(self):
        x0 = np.ravel(self.solution)
        self.res = minimize(self.loss_classification, x0, options = {'maxiter': 1})
        self.solution = self.res.x.reshape((self.vertex_num, 3))
    
    def optimize_test(self, num_iteration):
        for i in range(num_iteration):
            self.optimize_one_round()
            print(i)

In [5]:
test = chen_2023(vertices.copy(), faces.copy(), max_iter = 50)

In [6]:
test.optimize()

/tmp/ipykernel_129202/732141292.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  return np.power(np.cos(angle), 2) / angle


In [7]:
test.res

      fun: 427.9104714255486
 hess_inv: array([[ 1.02077653, -0.04439904, -0.10578906, ..., -0.02926349,
        -0.01845638,  0.16339046],
       [-0.04439904,  1.03310191,  0.10009986, ...,  0.03826582,
         0.02866771, -0.22717926],
       [-0.10578906,  0.10009986,  1.04441286, ...,  0.08390636,
         0.06135829, -0.47330906],
       ...,
       [-0.02926349,  0.03826582,  0.08390636, ...,  1.00944972,
         0.01464719, -0.10401916],
       [-0.01845638,  0.02866771,  0.06135829, ...,  0.01464719,
         1.01107298, -0.09241038],
       [ 0.16339046, -0.22717926, -0.47330906, ..., -0.10401916,
        -0.09241038,  1.68724756]])
      jac: array([-2.93752289e+00, -1.20415497e+00,  2.80121613e+00, -3.50063324e-01,
       -9.08699036e-01,  8.95553589e-01,  2.54961777e+00,  3.26140594e+00,
       -6.82987213e+00, -4.36107254e+00,  1.36455536e-01, -1.90759659e+00,
        7.81331635e+00,  1.97376251e-01, -7.76268005e-01,  1.49369240e+01,
        1.67863464e+00, -3.67922974e

In [8]:
mesh_update = pv.PolyData(test.solution, sphere.faces)

In [9]:
p = pv.Plotter()
p.add_mesh(mesh_update, show_edges=True, color="green", opacity=0.5)
p.add_mesh(sphere, show_edges=True, color="white", opacity=0.5)
p.add_points(mesh_update.points, color="red", point_size=5)
p.show()

Widget(value="<iframe src='http://localhost:37593/index.html?ui=P_0x7f85468e4910_1&reconnect=auto' style='widt…

In [11]:
ship = pv.read('input.ply')

In [12]:
faces = ship.faces.reshape((-1,4))[:, 1:4]
vertices = ship.points
angles = calculate_all_dihedral_angles(faces, vertices)
edges = retrieve_all_edges(faces)

In [13]:
vertices.shape

(256, 3)

In [14]:
faces.shape

(508, 3)

In [15]:
test_ship = chen_2023(vertices.copy(), faces.copy(), max_iter = 10)

In [16]:
test_ship.optimize_test(10)

0
1
2
3
4
5
6
7
8
9


In [18]:
test_ship_update = pv.PolyData(test_ship.solution, ship.faces)

In [17]:
test_ship.solution

array([[ 1.25260024e-01,  1.53389020e-02,  1.07594470e-01],
       [ 1.45500483e-01, -9.33043449e-02,  1.74015930e-01],
       [ 5.90096803e-02, -2.81831090e-02, -6.65109775e-03],
       [ 9.61305268e-02,  7.17430918e-03, -3.90111161e-05],
       [ 6.71026581e-02, -3.71035750e-02, -9.91225245e-02],
       [ 1.26912985e-01,  6.04981904e-03, -8.43856721e-02],
       [ 1.32790823e-01, -5.27599486e-02, -1.99868866e-01],
       [ 1.49764252e-01, -7.97591251e-03, -1.97632134e-01],
       [ 1.45440033e-01, -8.22876375e-02, -2.71796554e-01],
       [ 4.98154225e-02,  7.31940799e-02, -2.95979956e-01],
       [ 8.40881161e-02, -5.14261299e-02, -3.42200123e-01],
       [ 8.68017761e-02,  4.16521006e-02, -3.69756297e-01],
       [ 1.83439882e-01,  5.54821821e-03,  1.17091570e-01],
       [ 1.23316828e-01, -5.02198011e-02,  1.85282121e-01],
       [ 8.22012010e-02,  2.74289842e-02,  2.82197163e-01],
       [ 6.69721895e-02, -5.30396772e-02,  3.03888561e-01],
       [ 8.12011642e-02,  3.82374659e-02

In [23]:
p2 = pv.Plotter()
p2.add_mesh(test_ship_update, show_edges=True, color="green", opacity=0.5)
p2.add_mesh(ship, show_edges=True, color="white", opacity=0.5)
# p2.add_points(ship.points, color="red", point_size=5)
p2.show()

Widget(value="<iframe src='http://localhost:37593/index.html?ui=P_0x7f853c105540_6&reconnect=auto' style='widt…